In [1]:
import numpy as np
import pandas as pd

import nmslib
import json

import scipy.sparse as sparse
from lightfm import LightFM

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Загружаем датасеты
train = pd.read_csv('train.csv')
meta = pd.read_csv('meta.csv')

# Удалим дубликаты из тренировочного датасета
train.drop_duplicates(inplace = True)

# Объединим тренировочный датасет и данные из meta по идентификатору asin (Amazon Standard Identification Number)
df = pd.merge(train, meta, on='asin')
#df_new_test = pd.merge(test, meta, on='asin')

In [2]:
df = df[['userid','itemid','rating','main_cat', 'title']]

In [3]:
df.main_cat = df.main_cat.fillna('Other')

In [4]:
dic_main_cat = {
    'Grocery':1,
    'Health & Personal Care':2,
    'Amazon Home':3,
    'All Beauty':4,
    'Sports & Outdoors':5,
    'Other':6,
    'Industrial & Scientific':7,
    'Office Products':8,
    'Toys & Games':9,
    'Pet Supplies':10,
    'Tools & Home Improvement':11,
    'Baby':12,
    'Arts, Crafts & Sewing':13,
    'Home Audio & Theater':14,
    'Camera & Photo':15,
    'Musical Instruments':16,
    'Software':17,
    'Cell Phones & Accessories':18,
}
df['main_cat'] = df['main_cat'].map(dic_main_cat)

In [5]:
ratings_coo = sparse.coo_matrix((df.rating,(df.userid,df.itemid)))
feature_ratings  = sparse.coo_matrix(([1]*len(df),(df.itemid,df.main_cat)))

In [6]:
LR = 0.01
NUM_THREADS = 6 #число потоков
NUM_COMPONENTS = 50 #число параметров вектора 
NUM_EPOCHS = 10 #число эпох обучения
LOSS_FUNCTION = 'logistic'
LEARNING_SCHEDULE = 'adagrad'
RANDOM_STATE = 42

model = LightFM(
    learning_rate=LR,
    loss=LOSS_FUNCTION,
    no_components=NUM_COMPONENTS,
    learning_schedule = LEARNING_SCHEDULE,
    random_state = RANDOM_STATE
)

#Обучаем модель на всем тренировочном датасете т.к. на данном этапе нам важно быстрее получить модель и эмбеддинги для прототипа
model = model.fit(
    ratings_coo,
    epochs=NUM_EPOCHS,
    num_threads=NUM_THREADS,
    item_features=feature_ratings
)

In [7]:
# Достаём эбмеддинги
item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

In [8]:
#Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять наши книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [9]:
#Вспомогательная функция для поиска по графу
def nearest_items_nms(itemid, index, n=10):
    nn = index.knnQuery(item_embeddings[itemid], k=n)
    return nn

In [10]:
#Давайте попробуем написать рекомендации к продукту. Например, к  «1984».
df[df.title.str.find('Lipton Yellow Label Tea')>=0].title.unique()

array(['Lipton Yellow Label Tea Bags 100ct, 1 pack',
       'Lipton Yellow Label Tea (loose tea) - 450g',
       'Lipton Yellow Label Tea (loose tea)'], dtype=object)

In [11]:
#Видим, что у книги id — 846.
#Ищем похожие книги.
nbm = nearest_items_nms(846,nms_idx)[0]

In [12]:
#Выводим похожие книги.
df[df.itemid.isin(nbm)].title.unique()[0:4]

array(['SNICKERS Almond Sharing Size Chocolate Candy Bars 3.23-Ounce Bar 24-Count Box',
       'NOW Foods Beef Gelatin Powder,16-Ounce(Pack of 2)',
       'Green Mountain Coffee Hazelnut Decaf K-Cups for Keurig Brewers - 18 K-Cups',
       'Zsweet All Natural Zero Calorie Sweetener, 1.5-Pound Pouches (Pack of 2)'],
      dtype=object)

In [13]:
import pickle
with open('item_embeddings.pickle', 'wb') as file:
   pickle.dump(item_embeddings, file, protocol=3)

In [14]:
df = df[['itemid','title']]

In [15]:
df.drop_duplicates(inplace = True)

In [16]:
df.to_csv('df_items.csv', index=False)